In [36]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import sys, getopt
import csv
import time

In [37]:
def get_links(url):
    res = requests.get(url)
    ## The next two lines get around the issue with comments breaking the parsing.
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("", res.text), 'lxml')
    all_tables = soup.findAll("tbody")
    test = soup.find(id='div_sched_2020-2021_9_1')
    x = "score"
    test = test.findAll("td", {"data-stat": x})
    link_list = []
    for i in test:
        i = i.find("a")
        if i is not None:
            link = i['href']
            link_list.append(link)
            #print(link)
    return link_list

In [39]:
#@title

#stats matchday
statsm = ["player", "shirtnumber", "nationality", "position", "age", "minutes", "goals", "assists", "touches", "sca",
          "gca", 'pens_made', 'shots_total', 'shots_on_target', 'pressures', 'tackles', 'interceptions', 'blocks', 'xg',
          'xa', 'passes', 'passes_pct', 'progressive_passes', 'progressive_carries', 'dribbles_completed', 'dribbles']

#stats keeper matchday
statskeepers = ['player', 'minutes', 'age', 'gk_shots_on_target_against', 'gk_goals_against', 'gk_saves', 'gk_save_pct',
                'gk_psxg', 'gk_passes_completed_launched', 'gk_passes', 'gk_passes_throws', 'gk_pct_passes_launched',
                'gk_def_actions_outside_pen_area']

keepers = ["player", "nationality", "position", "squad", "age", "birth_year", "games_gk", "games_starts_gk",
           "minutes_gk", "goals_against_gk", "goals_against_per90_gk", "shots_on_target_against", "saves", "save_pct",
           "wins_gk", "draws_gk", "losses_gk", "clean_sheets", "clean_sheets_pct", "pens_att_gk", "pens_allowed",
           "pens_saved", "pens_missed_gk"]
keepers3 = ["players_used", "games_gk", "games_starts_gk", "minutes_gk", "goals_against_gk", "goals_against_per90_gk",
            "shots_on_target_against", "saves", "save_pct", "wins_gk", "draws_gk", "losses_gk", "clean_sheets",
            "clean_sheets_pct", "pens_att_gk", "pens_allowed", "pens_saved", "pens_missed_gk"]


#advance goalkeeping(keepersadv)


#Functions to get the data in a dataframe using BeautifulSoup

def get_tables_m(url):
    res = requests.get(url)
    ## The next two lines get around the issue with comments breaking the parsing.
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("", res.text), 'lxml')
    all_tables = soup.findAll("tbody")
    matchweek = soup.find(id='content').find("div").contents[1]
    date = soup.find("div", {"class": "scorebox_meta"}).find("div").contents[0].contents[0]
    date = date['href']
    date = date[12:22]
    #date = soup.find("div", {"class": "scorebox_meta"}).soup.find("span", {"class": "venuetime"})

    team_H = soup.findAll("strong")[2].find("a").contents[0]
    if soup.findAll("strong")[5].find("a") is None:
        team_A = soup.findAll("strong")[4].find("a").contents[0]
    else:
        team_A = soup.findAll("strong")[5].find("a").contents[0]

    team1_table = all_tables[0]
    team2_table = all_tables[7]
    gk1_table = all_tables[6]
    gk2_table = all_tables[13]

    #gk1_table.append(gk2_table)
    #team1_table.append(team2_table)

    return team1_table, team2_table, gk1_table, gk2_table, matchweek, date, team_H, team_A


def get_frame(features, player_table):
    pre_df_player = dict()
    features_wanted_player = features
    rows_player = player_table.find_all('tr')
    rows_player
    #print(rows_player)
    for row in rows_player:
        #print(row)
        if (row.find('th', {"scope": "row"}) != None):

            for f in features_wanted_player:
                #print(f)
                cell = row.find("td", {"data-stat": f})
                #print(cell)
                if cell is None:
                    text = row.find('th', {"scope": "row"}).find('a').contents[0]
                    if f in pre_df_player:
                        pre_df_player[f].append(text)
                    else:
                        pre_df_player[f] = [text]
                #print(cell)
                else:
                    a = cell.text.strip().encode()
                    text = a.decode("utf-8")
                    #print(text)
                    if (text == ''):
                        text = '0'
                    if ((f != 'player') & (f != 'nationality') & (f != 'position') & (f != 'squad') & (f != 'age') & (
                            f != 'birth_year')):
                        text = float(text.replace(',', ''))
                    if f in pre_df_player:
                        pre_df_player[f].append(text)
                    else:
                        pre_df_player[f] = [text]
    #print(pre_df_player)
    df_player = pd.DataFrame.from_dict(pre_df_player)
    #print(df_player)
    return df_player


def frame_for_category(category, top, end, features):
    url = (top + category + end)
    print(url)
    player_table_H, player_table_A, gk_table_H, gk_table_A, matchweek, date, team_H, team_A = get_tables_m(url)
    #print(player_table)
    df_player_H = get_frame(features, player_table_H)
    df_player_H['Team'] = team_H
    df_player_A = get_frame(features, player_table_A)
    df_player_A['Team'] = team_A
    df_player = pd.concat([df_player_H, df_player_A])
    df_player['matchweek'] = matchweek
    df_player['date'] = date
    df_gk_H = get_frame(statskeepers, gk_table_H)
    df_gk_H['Team'] = team_H
    df_gk_A = get_frame(statskeepers, gk_table_A)
    df_gk_A['Team'] = team_A
    df_gk = pd.concat([df_gk_H, df_gk_A])
    df_gk['matchweek'] = matchweek
    df_gk['date'] = date
    return df_player, df_gk


def get_matchday_data(top, end, linklist):
    pl, gk = frame_for_category('/en/matches/bf52349b', top, end, statsm)
    #pl1, gk1 = frame_for_category('/en/matches/3adf2aa7', top, end, statsm)

    for i in linklist:
        tmppl, tmpgk = frame_for_category(i, top, end, statsm)
        pl = pd.concat([pl, tmppl], axis=0)
        gk = pd.concat([gk, tmpgk], axis=0)
        time.sleep(5)

    return pl, gk


#pl, gk = frame_for_category('/en/matches/51fb894e', 'https://fbref.com/', '', statsm)
#gk

In [45]:
def get_lineups(url):
    pre_df_player = dict()
    res = requests.get(url)
    ## The next two lines get around the issue with comments breaking the parsing.
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("", res.text), 'lxml')
    all_tables = soup.findAll("tbody")
    date = soup.find("div", {"class": "scorebox_meta"}).find("div").contents[0].contents[0]
    date = date['href']
    date = date[12:22]
    matchweek = soup.find(id='content').find("div").contents[1]
    lineup = soup.find_all("div", {"class": "lineup"})
    lineup_H = lineup[0]
    lineup_A = lineup[1]

    rows_lineup_H = lineup_H.find_all('tr')
    rows_lineup_A = lineup_A.find_all('tr')
    lineup_total = [rows_lineup_H, rows_lineup_A]
    #print(rows_lineup)
    i = 0


    for rows_lineup in lineup_total:
        i = i+1
        for row in rows_lineup:

            if row.find('a') != None:
                #print(row.find('a').contents[0])
                cell = row.find('a').contents[0]
                a = cell.text.strip().encode()
                text = a.decode("utf-8")

                if 'player' in pre_df_player:
                    pre_df_player['player'].append(text)
                    #pre_df_player['count'].append(3)
                else:
                    pre_df_player['player'] = [text]
                    #pre_df_player['count'] = i

        if i == 1:
            df_player_H = pd.DataFrame.from_dict(pre_df_player)
        else: df_player_A = pd.DataFrame.from_dict(pre_df_player)
        pre_df_player = dict()



    team_H = soup.findAll("strong")[2].find("a").contents[0]
    if soup.findAll("strong")[5].find("a") is None:
        team_A = soup.findAll("strong")[4].find("a").contents[0]
    else:
        team_A = soup.findAll("strong")[5].find("a").contents[0]

    #gk1_table.append(gk2_table)
    #team1_table.append(team2_table)

    return df_player_H, df_player_A, team_H, team_A, matchweek, date

def frame_for_category_lineups (category, top, end):
    url = (top + category + end)
    print(url)
    player_H, player_A, teamH, teamA, matchweek, date,= get_lineups(url)

    player_H['team'] = teamH
    player_A['team'] = teamA
    df_player = pd.concat([player_H, player_A])
    df_player['matchweek'] = matchweek
    df_player['date'] = date
    df_player['start11'] = np.where(df_player.index< 11, True, False)

    return df_player

def get_matchday_data_lineups(top, end, linklist):
    pl = frame_for_category_lineups('/en/matches/bf52349b', top, end)
    #pl1, gk1 = frame_for_category('/en/matches/3adf2aa7', top, end, statsm)

    for i in linklist:
        tmppl = frame_for_category_lineups(i, top, end)
        pl = pd.concat([pl, tmppl], axis=0)
        time.sleep(5)

    return pl


#H, A, teamH, teamA, matchweek, date = get_lineups(
 #   'https://fbref.com/en/matches/4a652b81/Tottenham-Hotspur-West-Bromwich-Albion-February-7-2021#-Premier-League')
#frame_for_category_lineups('/en/matches/51fb894e', 'https://fbref.com/', '')

In [41]:
link_list = get_links('https://fbref.com/en/comps/9/10728/schedule/2020-2021-Premier-League-Scores-and-Fixtures')

# split list in 4 parts to not bring down website
list1 = []
list2 = []
list3 = []
list4 = []
count = 0

for i in link_list:
    if count < 100:
        list1.append(i)
    elif count < 200:
        list2.append(i)
    elif count < 300:
        list3.append(i)
    else:
        list4.append(i)
    count = count + 1

In [92]:

df_1, gk_1 = get_matchday_data('https://fbref.com/', '', list1)
df_2, gk_2 = get_matchday_data('https://fbref.com/', '', list2)
df_3, gk_3 = get_matchday_data('https://fbref.com/', '', list3)
df_4, gk_4 = get_matchday_data('https://fbref.com/', '', list4)

#df_test, gk_test = get_matchday_data('https://fbref.com/', '', link_list )

https://fbref.com//en/matches/bf52349b
https://fbref.com//en/matches/be1ee647/Fulham-Wolverhampton-Wanderers-April-9-2021-Premier-League
https://fbref.com//en/matches/ad969519/Manchester-City-Leeds-United-April-10-2021-Premier-League
https://fbref.com//en/matches/20bdebdc/Liverpool-Aston-Villa-April-10-2021-Premier-League
https://fbref.com//en/matches/54b07679/Crystal-Palace-Chelsea-April-10-2021-Premier-League
https://fbref.com//en/matches/daebaaa1/Burnley-Newcastle-United-April-11-2021-Premier-League
https://fbref.com//en/matches/06b2801a/West-Ham-United-Leicester-City-April-11-2021-Premier-League
https://fbref.com//en/matches/a51d567c/Tottenham-Hotspur-Manchester-United-April-11-2021-Premier-League
https://fbref.com//en/matches/34395b54/Sheffield-United-Arsenal-April-11-2021-Premier-League
https://fbref.com//en/matches/3dee3681/West-Bromwich-Albion-Southampton-April-12-2021-Premier-League
https://fbref.com//en/matches/ac6ac8f8/Brighton-and-Hove-Albion-Everton-April-12-2021-Premier-L

In [50]:
l_1 = get_matchday_data_lineups('https://fbref.com/', '', list1)
l_2 = get_matchday_data_lineups('https://fbref.com/', '', list2)
l_3 = get_matchday_data_lineups('https://fbref.com/', '', list3)
l_4 = get_matchday_data_lineups('https://fbref.com/', '', list4)

https://fbref.com//en/matches/bf52349b
https://fbref.com//en/matches/bf52349b/Fulham-Arsenal-September-12-2020-Premier-League
https://fbref.com//en/matches/db261cb0/Crystal-Palace-Southampton-September-12-2020-Premier-League
https://fbref.com//en/matches/21b58926/Liverpool-Leeds-United-September-12-2020-Premier-League
https://fbref.com//en/matches/78495ced/West-Ham-United-Newcastle-United-September-12-2020-Premier-League
https://fbref.com//en/matches/7dd01ca9/West-Bromwich-Albion-Leicester-City-September-13-2020-Premier-League
https://fbref.com//en/matches/fc7f9aa1/Tottenham-Hotspur-Everton-September-13-2020-Premier-League
https://fbref.com//en/matches/a3eb7a37/Sheffield-United-Wolverhampton-Wanderers-September-14-2020-Premier-League
https://fbref.com//en/matches/9d7641eb/Brighton-and-Hove-Albion-Chelsea-September-14-2020-Premier-League
https://fbref.com//en/matches/45bd1880/Everton-West-Bromwich-Albion-September-19-2020-Premier-League
https://fbref.com//en/matches/583c2b60/Leeds-Unite

In [51]:
l_1

,player,team,matchweek,date,start11
0,Marek Rodák,Fulham,(Matchweek 1),2020-09-12,True
1,Michael Hector,Fulham,(Matchweek 1),2020-09-12,True
2,Denis Odoi,Fulham,(Matchweek 1),2020-09-12,True
3,Neeskens Kebano,Fulham,(Matchweek 1),2020-09-12,True
4,Tom Cairney,Fulham,(Matchweek 1),2020-09-12,True
...,...,...,...,...,...
13,Tom Cairney,Fulham,(Matchweek 11),2020-12-05,False
14,Tim Ream,Fulham,(Matchweek 11),2020-12-05,False
15,Mario Lemina,Fulham,(Matchweek 11),2020-12-05,False
16,Joe Bryan,Fulham,(Matchweek 11),2020-12-05,False


In [52]:
def concatenate_files(f1, f2, f3, f4):
    total = pd.concat([f1, f2, f3, f4], axis=0)
    total = total.drop_duplicates()
    total.reset_index(inplace=True, drop=True)
    return total

In [94]:
gk_2020 = concatenate_files(gk_1, gk_2, gk_3, gk_4)
pl_2020 = concatenate_files(df_1, df_2, df_3, df_4)

In [53]:
lineups_2020 = concatenate_files(l_1, l_2, l_3, l_4)

In [95]:
pl_2020['matchweek'] = pl_2020['matchweek'].str.slice(start=-3, stop=-1).astype(float)
gk_2020['matchweek'] = gk_2020['matchweek'].str.slice(start=-3, stop=-1).astype(float)
pl_2020['position'] = pl_2020['position'].str.slice(start=0, stop=2)
gk_2020

,player,minutes,age,gk_shots_on_target_against,gk_goals_against,gk_saves,gk_save_pct,gk_psxg,gk_passes_completed_launched,gk_passes,gk_passes_throws,gk_pct_passes_launched,gk_def_actions_outside_pen_area,Team,matchweek,date
0,Marek Rodák,90.0,23-274,5.0,3.0,2.0,40.0,2.5,6.0,22.0,3.0,40.9,1.0,Fulham,1.0,2020-09-12
1,Bernd Leno,90.0,28-192,2.0,0.0,2.0,100.0,0.0,1.0,28.0,2.0,0.0,1.0,Arsenal,1.0,2020-09-12
2,Vicente Guaita,90.0,33-246,5.0,0.0,5.0,100.0,0.8,6.0,20.0,0.0,95.0,0.0,Crystal Palace,1.0,2020-09-12
3,Alex McCarthy,90.0,30-284,3.0,1.0,2.0,66.7,0.7,8.0,44.0,8.0,38.6,4.0,Southampton,1.0,2020-09-12
4,Alisson,90.0,27-346,3.0,3.0,0.0,0.0,2.1,2.0,26.0,5.0,34.6,0.0,Liverpool,1.0,2020-09-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760,Emiliano Martínez,90.0,28-263,6.0,1.0,5.0,83.3,1.2,9.0,24.0,4.0,62.5,0.0,Aston Villa,38.0,2021-05-23
761,Edouard Mendy,45.0,29-083,2.0,1.0,1.0,50.0,1.1,0.0,9.0,4.0,0.0,0.0,Chelsea,38.0,2021-05-23
762,Kepa Arrizabalaga,45.0,26-232,0.0,1.0,0.0,0.0,0.6,2.0,9.0,0.0,33.3,1.0,Chelsea,38.0,2021-05-23
763,Łukasz Fabiański,90.0,36-035,5.0,0.0,5.0,100.0,1.0,14.0,28.0,6.0,60.7,0.0,West Ham United,38.0,2021-05-23


In [54]:
lineups_2020['matchweek'] = lineups_2020['matchweek'].str.slice(start=-3, stop=-1).astype(float)

In [55]:
lineups_2020

,player,team,matchweek,date,start11
0,Marek Rodák,Fulham,1.0,2020-09-12,True
1,Michael Hector,Fulham,1.0,2020-09-12,True
2,Denis Odoi,Fulham,1.0,2020-09-12,True
3,Neeskens Kebano,Fulham,1.0,2020-09-12,True
4,Tom Cairney,Fulham,1.0,2020-09-12,True
...,...,...,...,...,...
14638,Michael Obafemi,Southampton,38.0,2021-05-23,False
14639,Nathan Tella,Southampton,38.0,2021-05-23,False
14640,Theo Walcott,Southampton,38.0,2021-05-23,False
14641,Dan Nlundulu,Southampton,38.0,2021-05-23,False


In [96]:
gk_2020['position'] = 'GK'

In [97]:
gk_2020

,player,minutes,age,gk_shots_on_target_against,gk_goals_against,gk_saves,gk_save_pct,gk_psxg,gk_passes_completed_launched,gk_passes,gk_passes_throws,gk_pct_passes_launched,gk_def_actions_outside_pen_area,Team,matchweek,date,position
0,Marek Rodák,90.0,23-274,5.0,3.0,2.0,40.0,2.5,6.0,22.0,3.0,40.9,1.0,Fulham,1.0,2020-09-12,GK
1,Bernd Leno,90.0,28-192,2.0,0.0,2.0,100.0,0.0,1.0,28.0,2.0,0.0,1.0,Arsenal,1.0,2020-09-12,GK
2,Vicente Guaita,90.0,33-246,5.0,0.0,5.0,100.0,0.8,6.0,20.0,0.0,95.0,0.0,Crystal Palace,1.0,2020-09-12,GK
3,Alex McCarthy,90.0,30-284,3.0,1.0,2.0,66.7,0.7,8.0,44.0,8.0,38.6,4.0,Southampton,1.0,2020-09-12,GK
4,Alisson,90.0,27-346,3.0,3.0,0.0,0.0,2.1,2.0,26.0,5.0,34.6,0.0,Liverpool,1.0,2020-09-12,GK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760,Emiliano Martínez,90.0,28-263,6.0,1.0,5.0,83.3,1.2,9.0,24.0,4.0,62.5,0.0,Aston Villa,38.0,2021-05-23,GK
761,Edouard Mendy,45.0,29-083,2.0,1.0,1.0,50.0,1.1,0.0,9.0,4.0,0.0,0.0,Chelsea,38.0,2021-05-23,GK
762,Kepa Arrizabalaga,45.0,26-232,0.0,1.0,0.0,0.0,0.6,2.0,9.0,0.0,33.3,1.0,Chelsea,38.0,2021-05-23,GK
763,Łukasz Fabiański,90.0,36-035,5.0,0.0,5.0,100.0,1.0,14.0,28.0,6.0,60.7,0.0,West Ham United,38.0,2021-05-23,GK


In [98]:
#map position to category
def get_category(df):
    df.loc[df['position'] == 'GK', 'category'] = 'Goalkeeper'
    df.loc[df['position'] == 'DF', 'category'] = 'Defender'
    df.loc[df['position'] == 'RB', 'category'] = 'Defender'
    df.loc[df['position'] == 'LB', 'category'] = 'Defender'
    df.loc[df['position'] == 'CB', 'category'] = 'Defender'
    df.loc[df['position'] == 'WB', 'category'] = 'Defender'
    df.loc[df['position'] == 'FB', 'category'] = 'Defender'
    df.loc[df['position'] == 'MF', 'category'] = 'Midfield'
    df.loc[df['position'] == 'DM', 'category'] = 'Midfield'
    df.loc[df['position'] == 'CM', 'category'] = 'Midfield'
    df.loc[df['position'] == 'AM', 'category'] = 'Midfield'
    df.loc[df['position'] == 'LM', 'category'] = 'Midfield'
    df.loc[df['position'] == 'RM', 'category'] = 'Midfield'
    df.loc[df['position'] == 'WM', 'category'] = 'Midfield'
    df.loc[df['position'] == 'FW', 'category'] = 'Forward'
    df.loc[df['position'] == 'RW', 'category'] = 'Forward'
    df.loc[df['position'] == 'LW', 'category'] = 'Forward'
    return df


pl_2020 = get_category(pl_2020)

In [56]:
def create_csv(df):
    df = df.drop_duplicates()
    df.reset_index(inplace=True, drop=True)
    df.to_csv('PL2020_lineups.csv', index=False)
    return df


create_csv(lineups_2020)


,player,team,matchweek,date,start11
0,Marek Rodák,Fulham,1.0,2020-09-12,True
1,Michael Hector,Fulham,1.0,2020-09-12,True
2,Denis Odoi,Fulham,1.0,2020-09-12,True
3,Neeskens Kebano,Fulham,1.0,2020-09-12,True
4,Tom Cairney,Fulham,1.0,2020-09-12,True
...,...,...,...,...,...
14638,Michael Obafemi,Southampton,38.0,2021-05-23,False
14639,Nathan Tella,Southampton,38.0,2021-05-23,False
14640,Theo Walcott,Southampton,38.0,2021-05-23,False
14641,Dan Nlundulu,Southampton,38.0,2021-05-23,False


In [100]:
def create_csv(df):
    df = df.drop_duplicates()
    df.reset_index(inplace=True, drop=True)
    df.to_csv('PL2020_keepers.csv', index=False)
    return df


create_csv(gk_2020)


,player,minutes,age,gk_shots_on_target_against,gk_goals_against,gk_saves,gk_save_pct,gk_psxg,gk_passes_completed_launched,gk_passes,gk_passes_throws,gk_pct_passes_launched,gk_def_actions_outside_pen_area,Team,matchweek,date,position
0,Marek Rodák,90.0,23-274,5.0,3.0,2.0,40.0,2.5,6.0,22.0,3.0,40.9,1.0,Fulham,1.0,2020-09-12,GK
1,Bernd Leno,90.0,28-192,2.0,0.0,2.0,100.0,0.0,1.0,28.0,2.0,0.0,1.0,Arsenal,1.0,2020-09-12,GK
2,Vicente Guaita,90.0,33-246,5.0,0.0,5.0,100.0,0.8,6.0,20.0,0.0,95.0,0.0,Crystal Palace,1.0,2020-09-12,GK
3,Alex McCarthy,90.0,30-284,3.0,1.0,2.0,66.7,0.7,8.0,44.0,8.0,38.6,4.0,Southampton,1.0,2020-09-12,GK
4,Alisson,90.0,27-346,3.0,3.0,0.0,0.0,2.1,2.0,26.0,5.0,34.6,0.0,Liverpool,1.0,2020-09-12,GK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760,Emiliano Martínez,90.0,28-263,6.0,1.0,5.0,83.3,1.2,9.0,24.0,4.0,62.5,0.0,Aston Villa,38.0,2021-05-23,GK
761,Edouard Mendy,45.0,29-083,2.0,1.0,1.0,50.0,1.1,0.0,9.0,4.0,0.0,0.0,Chelsea,38.0,2021-05-23,GK
762,Kepa Arrizabalaga,45.0,26-232,0.0,1.0,0.0,0.0,0.6,2.0,9.0,0.0,33.3,1.0,Chelsea,38.0,2021-05-23,GK
763,Łukasz Fabiański,90.0,36-035,5.0,0.0,5.0,100.0,1.0,14.0,28.0,6.0,60.7,0.0,West Ham United,38.0,2021-05-23,GK


In [129]:
#gk_2021[['player', 'matchweek']]

test_group = gk_2021.groupby(['player']).sum()['saves']
gk_strength = test_group.sort_values(ascending=False)
gk_strength

player
Illan Meslier          130.0
David de Gea           122.0
José Sá                119.0
Kasper Schmeichel      113.0
Jordan Pickford        107.0
Nick Pope               98.0
Robert Sánchez          96.0
Łukasz Fabiański        95.0
Tim Krul                93.0
Hugo Lloris             90.0
Emiliano Martínez       84.0
Vicente Guaita          79.0
Aaron Ramsdale          77.0
David Raya              68.0
Edouard Mendy           67.0
Fraser Forster          67.0
Martin Dúbravka         66.0
Alisson                 65.0
Ben Foster              56.0
Ederson                 54.0
Álvaro Fernández        46.0
Daniel Bachmann         39.0
Alex McCarthy           38.0
Angus Gunn              26.0
Karl Darlow             25.0
Kepa Arrizabalaga       11.0
Jack Butland            11.0
Bernd Leno               9.0
Freddie Woodman          9.0
Asmir Begović            7.0
Wayne Hennessey          7.0
Willy Caballero          6.0
Caoimhín Kelleher        5.0
Kristoffer Klaesson      4.0
Jason S

In [133]:
rslt_df = gk_2021[gk_2021['player'] == 'Bernd Leno']
rslt_df[['player', 'matchweek']]

,player,matchweek
1,Bernd Leno,(Matchweek 1)
36,Bernd Leno,(Matchweek 2)
41,Bernd Leno,(Matchweek 3)
573,Bernd Leno,(Matchweek 30)
